In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN_Wmultitask_16_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 6    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 2, 2]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
train_data.shape

(691, 1024, 77)

In [10]:
pos_weight.shape

(19,)

In [11]:
pos_weight

array([  0.36565482,  48.46756152,  47.65126513,  50.0669746 ,
        49.94930876,  40.02411874,  41.52307692,  58.84303112,
        63.4664723 ,  75.37996546,  86.74603175,  80.14495413,
        87.27145709,  69.42038217,  68.86413902,  43.71587462,
        13.05721551,  58.2021419 ,  10.05323669])

In [12]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/checkpoint' 

In [14]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [15]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation,
                                 trainable_wavelet = True,
                                 trainable_conv = True,
                                 trainable_last = True,
                                 variables_weights = variables_weights)

In [16]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [17]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.065 vs. 0.060
F1 = 0.100 vs. 0.090
Accuracy = 0.013 vs. 0.056
CHANGE DETECTION
AUC = 0.087 vs. 0.081
F1 = 0.158 vs. 0.159
Precision = 0.086 vs. 0.086
Recall = 0.998 vs. 0.996
Validation
ACTIVITY RECOGNITION
AUC = 0.062 vs. 0.059
F1 = 0.086 vs. 0.080
Accuracy = 0.003 vs. 0.056
CHANGE DETECTION
AUC = 0.049 vs. 0.048
F1 = 0.096 vs. 0.097
Precision = 0.051 vs. 0.051
Recall = 0.995 vs. 0.982
Minibatch Loss= 1.323188
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.540 vs. 0.061
F1 = 0.563 vs. 0.091
Accuracy = 0.581 vs. 0.056
CHANGE DETECTION
AUC = 0.210 vs. 0.099
F1 = 0.321 vs. 0.176
Precision = 0.225 vs. 0.097
Recall = 0.559 vs. 0.920
Validation
ACTIVITY RECOGNITION
AUC = 0.326 vs. 0.059
F1 = 0.412 vs. 0.082
Accuracy = 0.559 vs. 0.056
CHANGE DETECTION
AUC = 0.105 vs. 0.047
F1 = 0.192 vs. 0.096
Precision = 0.125 vs. 0.051
Recall = 0.416 vs. 0.987
Minibatch Loss= 0.400831
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.663 vs. 0.063


In [18]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/DWN_Wmultitask_16_opp/checkpoint-20000'

In [19]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../../model/DWN_Wmultitask_16_opp/checkpoint-20000')


In [20]:
#This is the model evaluation on test dataset
result = nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.627 vs. 0.059
F1 = 0.636 vs. 0.083
Accuracy = 0.907 vs. 0.056
CHANGE DETECTION
AUC = 0.206 vs. 0.049
F1 = 0.283 vs. 0.096
Precision = 0.208 vs. 0.051
Recall = 0.442 vs. 0.997


In [21]:
out = nn_eval.deploy(model, sess, test_data, test_gt)

In [22]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(out, axis = 2)

In [24]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})